<a href="https://colab.research.google.com/github/JeffersonEspinalA/Prueba/blob/main/auida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow
import numpy as np
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='sigmoid'))

In [ ]:
data=pd.read_csv('drive/MyDrive/HAM10000/HAM10000_metadata.csv')
data['image_full_name']=data['image_id']+'.jpg'
X=data[['image_full_name','dx','lesion_id']]

In [ ]:
X.sample(5)

,image_full_name,dx,lesion_id
3887,ISIC_0025822.jpg,nv,HAM_0004404
474,ISIC_0024381.jpg,bkl,HAM_0001057
3259,ISIC_0027912.jpg,nv,HAM_0002322
8772,ISIC_0028400.jpg,nv,HAM_0006118
6529,ISIC_0027690.jpg,nv,HAM_0004999


In [ ]:
from sklearn.model_selection import train_test_split
Y=X.pop('dx').to_frame()
X_train, X_test, y_train, y_test   = train_test_split(X,Y, test_size=0.17, random_state=42)
X_train,X_val,y_train,y_val        =train_test_split(X_train, y_train, test_size=0.17, random_state=42)

In [ ]:
train=pd.concat([X_train,y_train],axis=1)
val=pd.concat([X_val,y_val],axis=1)
test=pd.concat([X_test,y_test],axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
encoder.fit(val['dx'])
name_as_indexes_train=encoder.transform(val['dx']) 
val['label']=name_as_indexes_train

In [ ]:
encoder=LabelEncoder()
encoder.fit(test['dx'])
name_as_indexes_test=encoder.transform(test['dx']) 
test['label']=name_as_indexes_test

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale = 1./255,
                                     rotation_range=10,  
                                     zoom_range = 0.1, 
                                     width_shift_range=0.0,  height_shift_range=0.00) 

In [ ]:
train_data= train_generator.flow_from_dataframe(dataframe=train,x_col="image_full_name",y_col="dx",
                                                batch_size=32,directory="drive/MyDrive/HAM10000/HAM10000_images",
                                                shuffle=True,class_mode="categorical",target_size=(224,224))

Found 6898 validated image filenames belonging to 7 classes.


In [ ]:
test_generator=ImageDataGenerator(rescale = 1./255,rotation_range=10,  
                                     zoom_range = 0.1, 
                                     width_shift_range=0.0,  height_shift_range=0.02)

In [ ]:
test_data= test_generator.flow_from_dataframe(dataframe=test,x_col="image_full_name",y_col="dx",
                                              directory="drive/MyDrive/HAM10000/HAM10000_images",
                                              shuffle=False,batch_size=1,class_mode=None,target_size=(224,224))

Found 1703 validated image filenames.


In [ ]:
val_data=test_generator.flow_from_dataframe(dataframe=val,x_col="image_full_name",y_col="dx",
                                            directory="drive/MyDrive/HAM10000/HAM10000_images",
                                            batch_size=64,shuffle=False,class_mode="categorical",target_size=(224,224))

Found 1414 validated image filenames belonging to 7 classes.


In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_control = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=.5, min_lr=0.0001)

In [ ]:
from keras.optimizers import Adam,SGD
sgd = optimizers.SGD(lr = 0.01, clipvalue = 0.5)
model.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
history = model.fit_generator(generator=train_data,
                            steps_per_epoch=train_data.samples//train_data.batch_size,
                            validation_data=val_data,
                            verbose=1,
                            validation_steps=val_data.samples//val_data.batch_size,
                            epochs=3,callbacks=[learning_control])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/3
215/215 [==============================] - ETA: 0s - loss: 1.0833 - accuracy: 0.6708

215/215 [==============================] - 1279s 6s/step - loss: 1.0833 - accuracy: 0.6708 - val_loss: 0.9981 - val_accuracy: 0.6733 - lr: 0.0100
Epoch 2/3
215/215 [==============================] - ETA: 0s - loss: 1.0004 - accuracy: 0.6703

215/215 [==============================] - 439s 2s/step - loss: 1.0004 - accuracy: 0.6703 - val_loss: 0.9575 - val_accuracy: 0.6733 - lr: 0.0100
Epoch 3/3
215/215 [==============================] - ETA: 0s - loss: 0.9597 - accuracy: 0.6701

215/215 [==============================] - 430s 2s/step - loss: 0.9597 - accuracy: 0.6701 - val_loss: 0.9364 - val_accuracy: 0.6733 - lr: 0.0100


In [ ]:
test_data.reset()
predictions = model.predict_generator(test_data, steps=test_data.samples/test_data.batch_size,verbose=1)
y_pred= np.argmax(predictions, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


1703/1703 [==============================] - 312s 183ms/step


In [ ]:
c=np.where(y_pred == name_as_indexes_test)

In [ ]:
print("Test Accuracy :",(np.count_nonzero(c)/len(name_as_indexes_test))*100)

Test Accuracy : 66.23605402231357


In [ ]:
import tensorflow as tf

lesion_classes_dict = {
    0 : 'Melanocytic nevi',
    1 : 'Melanoma',
    2 : 'Benign keratosis-like lesions ',
    3 : 'Basal cell carcinoma',
    4 : 'Actinic keratoses',
    5 : 'Vascular lesions',
    6 : 'Dermatofibroma'
}

img = tf.keras.preprocessing.image.load_img("drive/MyDrive/HAM10000/HAM10000_images_part_1/ISIC_0024318.jpg", target_size=(224,224,3))
  
    #img = np.asarray(pil_image.open('img').resize((120,90)))
    #x = np.asarray(img.tolist())
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    #x = preprocess_input(x, mode='caffe')

preds = model.predict(x)
pred_class = preds.argmax(axis=-1)            # Simple argmax
        #pred_class = decode_predictions(preds, top=1)   
pr = lesion_classes_dict[pred_class[0]]
result =str(pr)  
print(result)   

1/1 [==============================] - 0s 45ms/step
Melanoma
